In [57]:
# Tested with python 3.10 and Qiskit 0.44
import numpy as np
try: # qiskit 1.0
    from qiskit.circuit.library import UnitaryGate
except ImportError: # Qiskit 0.44
    print("Import of UnitaryGate from Qiskit 1.0 location failed, will try to import as in Qiskit 0.44")
    from qiskit.extensions import UnitaryGate
    print("Import succeeded")
    
# UnitaryGate might also be found here, depending on your Qiskit version: 
# from qiskit.circuit.library.generalized_gates.unitary import UnitaryGate

from qiskit import QuantumCircuit, transpile
from qiskit.quantum_info import Operator, random_unitary
import cmath

np.set_printoptions(linewidth=10000)

DEBUG = False # setting to True will check (and print) intermediate decomposition results

Import of UnitaryGate from Qiskit 1.0 location failed, will try to import as in Qiskit 0.44
Import succeeded


### Helper functions

In [58]:
# Pretty printing the gate counts
# https://stackoverflow.com/questions/44689546/how-to-print-out-a-dictionary-nicely-in-python
def pretty_print_dict(dct):
    for item, amount in dct.items():  # dct.iteritems() in Python 2
        print("\t{}: {}".format(item, amount))

# Making matrices that are (almost) unitary into unitary gates
# This is necessary because the Qiskit unitary check is very strict, 
# and the decomposition introduces some small inaccuracies because of number precision etc.
# https://github.com/Qiskit/qiskit/issues/7120#issuecomment-940768863
def closest_unitary(A):
    """ Calculate the unitary matrix U that is closest with respect to the
        operator norm distance to the general matrix A.

        Return U as a numpy matrix.
    """
    # Needed because qiskit unitary check is too strict for the matrices at this point
    V, __, Wh = np.linalg.svd(A)
    U = np.matrix(V.dot(Wh))
    if not np.allclose(U, A):
         print("Warning: closest_unitary resulted in a matrix that is not close enough.")
    return U

### Helper gate definitions

In [59]:
def Rx(theta):
    return np.matrix([[np.cos(theta/2), -1j*np.sin(theta/2)],[-1j*np.sin(theta/2),np.cos(theta/2)]])

def Ry(theta):
    return np.matrix([[np.cos(theta/2), -np.sin(theta/2)],[np.sin(theta/2),np.cos(theta/2)]]) # This matches QX definition

def Rz(theta):
    return np.matrix([[np.exp(-1j*theta/2), 0],[0, np.exp(1j*theta/2)]]) 

### Decomp of U(4) up to diagonal (functions copied from Qiskit internals)
Used for optimization A2

In [60]:
def u4_to_su4(u4):
    phase_factor = np.conj(np.linalg.det(u4) ** (-1 / u4.shape[0]))
    su4 = u4 / phase_factor
    return su4, cmath.phase(phase_factor)

def cx2_test(u4):
    sy = np.array([[0, -1j], [1j, 0]])
    sysy = np.kron(sy, sy)
    su4,_ = u4_to_su4(u4)
    gamma = su4 @ sysy @ su4.T @ sysy
    # proposition III.3: two cx sufficient
    return(np.isclose(np.trace(gamma).imag, 0))
    
def real_trace_transform(mat):
    """
    Determine diagonal gate such that

    U3 = D U2

    Where U3 is a general two-qubit gate which takes 3 cnots, D is a
    diagonal gate, and U2 is a gate which takes 2 cnots.
    """
    a1 = (
        -mat[1, 3] * mat[2, 0]
        + mat[1, 2] * mat[2, 1]
        + mat[1, 1] * mat[2, 2]
        - mat[1, 0] * mat[2, 3]
    )
    a2 = (
        mat[0, 3] * mat[3, 0]
        - mat[0, 2] * mat[3, 1]
        - mat[0, 1] * mat[3, 2]
        + mat[0, 0] * mat[3, 3]
    )
    theta = 0  # arbitrary
    phi = 0  # arbitrary
    psi = np.arctan2(a1.imag + a2.imag, a1.real - a2.real) - phi
    diag = np.diag(np.exp(-1j * np.array([theta, phi, psi, -(theta + phi + psi)])))
    return diag

def decomp_u4_up_to_diagonal(mat):
    su4, phase = u4_to_su4(mat)
    real_map = real_trace_transform(su4)
    mapped_su4 = real_map @ su4
    if not cx2_test(mapped_su4):
          print("Warning: Unitary decomposition up to diagonal may use an additionl CX gate.")
    return real_map.conj(), mapped_su4, phase



## First round of decomposition: Block-ZXZ decomposition

In [61]:
def blockZXZ(mat, name=''):
    U = np.matrix(mat)
    n = mat.shape[0]
    n_half = int(0.5*n)
    I = np.identity(n_half)

    # upper left block
    X = U[0:n_half,0:n_half]
    # lower left block
    U12 = U[0:n_half,n_half:n]

    # svd: M = W*diag(S)*V.H
    # X = W11*diag(S11)*Vh11
    # X = Sx*Ux
    W11, S11, Vh11 = np.linalg.svd(X)
    Sx = W11 @ np.diag(S11) @ W11.conj().T 
    Ux = W11 @ Vh11

    # svd: U12 = W12*diag(S12)*Vh12
    # Y = Sy*Uy
    W12, S12, Vh12 = np.linalg.svd(U12)
    Sy = W12 @ np.diag(S12) @ W12.conj().T
    Uy = W12 @ Vh12

    A = np.matrix((Sx + 1j*Sy)*Ux)
    C = -1j*Ux.H*Uy
    B = U[n_half:n, 0:n_half]+U[n_half:n, n_half:n]*C.H
    Z = 2*A.H*X-I
    
    if DEBUG: # Used when debugging to check intermediate results
        IC = np.matrix(np.zeros((n,n), np.cdouble))
        IC[:n_half,:n_half] = I
        IC[n_half:,n_half:] = C

        AB = np.matrix(np.zeros((n,n), np.cdouble))
        AB[:n_half,:n_half] = A
        AB[n_half:n,n_half:n] = B

        IZ = np.matrix(np.zeros((n,n), np.cdouble))
        IZ[:n_half,:n_half] = I+Z
        IZ[n_half:,n_half:] = I+Z
        IZ[n_half:,:n_half] = I-Z
        IZ[:n_half,n_half:] = I-Z
        
        print('Block-ZXZ correct for matrix ', name, ' of shape: ', mat.shape, ' ? ', np.allclose(0.5*AB*IZ*IC, U))

    return A, B, Z, C

## Second round of decomposition: Demultiplexing

In [62]:
def demultiplex(M1, M2):
    eigenvalues, V = np.linalg.eig(M1*M2.H)
    D_sqrt = np.sqrt(eigenvalues)
    W = np.diag(D_sqrt) @ np.matrix(V).H @ M2
    if DEBUG: # Used when debugging to check intermediate results
        print('Demultiplexing correct? ', np.allclose(V @ np.diag(D_sqrt) @ W, M1), np.allclose(V @ np.diag(D_sqrt).conj() @ W, M2))
    return V, D_sqrt, W

# Decomposition of uniformly controlled Rz gates
Returns the normal or the inverse (reverse) Qiskit circuit corresponding to a (diagonal) matrix  of which the elements on the diagonal are the elements of array `D`.

Starts with the needed helper functions for Gray code etc.

In [63]:

#returns bitwise inner product
# uses https://stackoverflow.com/questions/109023/count-the-number-of-set-bits-in-a-32-bit-integer#109025
def bitwise_inner_product(a, b):
     i = a & b
     # number of set bits in i
     i = i - ((i >> 1) & 0x55555555)
     i = (i & 0x33333333) + ((i >> 2) & 0x33333333)
     i = (((i + (i >> 4)) & 0x0F0F0F0F) * 0x01010101) >> 24
     return i % 2

#returns M^k = (-1)^(b_(i-1)*g_(i-1)), where * is bitwise inner product, g = binary gray code, b = binary code.
def genMk(n):
	Mk = np.empty((n,n), int)
	for i in range(0,n):
		for j in range(0,n):
			Mk[i,j] = (-1)**(bitwise_inner_product((i),((j)^(j)>>1)))
	return(Mk)


# input D as (one dimensional) array
def cz_qiskit(D, circ_for_cz, controlbits, target, inverse=False):
    ar = np.linalg.solve(genMk(len(D)),(-2*np.log(D)/1j)) # solve for gray code ordering of the CNOTS with the natural logarithm of the eigenvalues of D.
	
    n = np.shape(D)[0]
    if(n != 2**len(controlbits)):
         print("Warning: shape mismatch for controlled Z between # control bits and length of D")

    if inverse: # inverse order of CNOTs and rotations 
        for i in range(n-1,-1,-1):
            idx = (i^(i>>1)^(i+1)^((i+1)>>1)).bit_length()-1 # first set bit
            if idx == target:
                posc = controlbits[-1]
            else:
                posc =  idx 
            circ_for_cz.cx(posc, target)
            circ_for_cz.rz(ar[i].real, target)
        
    else: # regular order of CNOTs and rotations 
        for i in range(0,n):
            idx = (i^(i>>1)^(i+1)^((i+1)>>1)).bit_length()-1 # first set bit
            if idx == target:
                posc = controlbits[-1]
            else:
                posc = idx 
            circ_for_cz.rz(ar[i].real, target)
            circ_for_cz.cx(posc, target)


## Optimization A2 (decomposing the U(4)s up to diagonal)
Based on _apply_a2 in Qiskit qsd.py

In [64]:
def apply_a2(circ):
    ccirc = transpile(circ, basis_gates=["u", "cx", "vbk2q"], optimization_level=0)
    ind2q = []
    # collect 2q instrs
    for i, instruction in enumerate(ccirc.data):
        if instruction.operation.name == "vbk2q":
            ind2q.append(i)
    if len(ind2q) == 0:
        return ccirc
    elif len(ind2q) == 1:
        # No neighbors to merge diagonal into; revert name
        ccirc.data[ind2q[0]].operation.name = "Unitary"
        return ccirc
    # rolling over diagonals
    ind2 = None  # lint
    for ind1, ind2 in zip(ind2q[0:-1:], ind2q[1::]):
        # get neigboring 2q gates separated by controls
        instr1 = ccirc.data[ind1]
        mat1 = Operator(instr1.operation).data
        instr2 = ccirc.data[ind2]
        mat2 = Operator(instr2.operation).data
        # rollover
        dmat, qc2cx, phase = decomp_u4_up_to_diagonal(mat1)
        
        if not np.allclose(np.matrix(qc2cx).H @ qc2cx, np.identity(4)):
             print("qc2cx not unitary")

        ccirc.data[ind1] = instr1.replace(operation=UnitaryGate(closest_unitary(qc2cx)))
        mat2 = mat2 @ dmat
        if (not np.allclose(np.matrix(mat2).H @ mat2, np.identity(4))):
             print("qc2cx not unitary")

        ccirc.data[ind2] = instr2.replace(UnitaryGate(mat2))
        ccirc.global_phase += phase
    ccirc.data[ind2] = ccirc.data[ind2].replace(operation=UnitaryGate(mat2))
    return ccirc

# Full decomposition
Starts with the recursive function, and then applies the A2 optimization

In [65]:
def decompose_recursion(mat, nqubits):

    if nqubits == 1:
        ugate = UnitaryGate(mat)
        qc = QuantumCircuit(1, name="vbk1q")
        qc.append(ugate, [0])
        return qc
    
    if nqubits == 2:
        if not np.allclose(mat.H*mat, np.identity(4)):
             # Needed because we run into Qiskit precision mismatches, so the unitary check is too strict
            if not np.allclose(mat.H*mat, np.identity(4), rtol=0.001, atol=0.001):
                print("2-qubit matrix not unitary") 
            else:
                # closest_unitary(mat) transforms the unitary to the closest unitary with a higher precision, so we can pass the checks again
                print("2-qubit matrix not unitary to high enough precision") 
                mat = closest_unitary(mat)
        ugate = UnitaryGate(mat)
        qc = QuantumCircuit(2, name="vbk2q")
        qc.append(ugate, [0, 1])
        return qc
    
    A1, A2, B, C = blockZXZ(mat)

    n = mat.shape[0]
    n_half = int(0.5*n)
    I = np.identity(n_half)

    V_a, D_a, W_a = demultiplex(A1,A2)
    V_c, D_c, W_c = demultiplex(I,C)

    circ = QuantumCircuit(nqubits)

    circ_W_c = decompose_recursion(W_c, nqubits-1)
    circ.append(circ_W_c.to_instruction(), range(nqubits-1))

    cz_qiskit(D_c, circ, range(nqubits-1), nqubits-1)
    del circ.data[-1] # remove the last cnot we added

    circ.h(nqubits-1)
    
    # upper left of the middle matrix
    new_middle_ul = W_a*V_c 

    # lower right of the middle matrix
    # https://quantumcomputing.stackexchange.com/questions/4252/how-to-derive-the-cnot-matrix-for-a-3-qubit-system-where-the-control-target-qu 
    Z_gate = [[1, 0], [0, -1]]
    new_middle_lr = np.kron(Z_gate, np.identity(int(0.25*n)))*W_a*B*V_c*np.kron(Z_gate, np.identity(int(0.25*n))) # lower right block

    V_m, D_sqrt_m, W_m = demultiplex(new_middle_ul, new_middle_lr)

    W_m_circ = decompose_recursion(W_m, nqubits-1)
    circ.append(W_m_circ.to_instruction(), range(nqubits-1))

    cz_qiskit(D_sqrt_m, circ, range(nqubits-1), nqubits-1)

    V_m_circ = decompose_recursion(V_m, nqubits-1)
    circ.append(V_m_circ.to_instruction(), range(nqubits-1))

    circ.h(nqubits-1)

    cz_qiskit(D_a, circ, range(nqubits-1), nqubits-1, inverse=True)
    del circ.data[-2**nqubits] # remove first cnot added with the last operation (the one next to middle gate)

    V_a_circ = decompose_recursion(V_a, nqubits-1)
    circ.append(V_a_circ.to_instruction(), range(nqubits-1))

    return circ
    
# full decomposition (upper level non-recursive function)
def decompose(matrix_init):
    nqubits = int(np.log2(matrix_init.shape[0]))
    if DEBUG:
        print("Decomposing matrix with: ", nqubits, " qubits")

    # First do the general decomposition up to the two qubit gates:
    circ_decompose_1 = decompose_recursion(matrix_init, nqubits)
    if DEBUG:
        print("Decomposition correct before optimization a2?: ", np.allclose(Operator(circ_decompose_1).data, matrix_init))

    # now we transform all the two qubit unitaries so they can be decomposed using less CNOTs (except the last one)
    circ_decompose_opt = apply_a2(circ_decompose_1)

    if DEBUG:
        print("Decomposition after optimization correct?: ", np.allclose(Operator(circ_decompose_opt).data, matrix_init))
    return circ_decompose_opt


## Running and verification of the decomposition
Decomposes a randomly generated unitary matrix.

The matrix is generated using Qiskit, the number of qubits can be changed to anything you want (so long as it is a whole number higher than 0)

In [66]:
# generating a random unitary using Qiskit
nqubits = 3
matrix_init = random_unitary(2**nqubits).data

# full decomposition (Qiskit does not automatically decompose the two qubit gates)
circ_decomposed = decompose(matrix_init)

#verification of the decomposition:
matrix_from_circ = Operator(circ_decomposed).data
print("Decomposition correct?: ", np.allclose(matrix_from_circ, matrix_init))
print("\nResulting gate count (u = generic one qubit gate, unitary = generic two qubit gate, cx = CNOTs needed for the decomposition of the multi-controlled Rz gates):\n\t", circ_decomposed.count_ops())


# decompose circuit to generic single qubit gates and CNOTs (and no further optimization):
circ_u3_cx = transpile(circ_decomposed, basis_gates=["u3", "cx"], optimization_level=0)
print("\nDecomposition correct after decomposing the two qubit gates?: ", np.allclose(Operator(circ_u3_cx).data, matrix_init))

print("Resulting gatecount after decomposing the two qubit gates to generic single qubit gates (u3) and CNOT (cx):")
pretty_print_dict(circ_u3_cx.count_ops())

# uncomment to print the circuit (not recommended for anything over 4 qubits)
# print(circ_u3_cx)

Decomposition correct?:  True

Resulting gate count (u = generic one qubit gate, unitary = generic two qubit gate, cx = CNOTs needed for the decomposition of the multi-controlled Rz gates):
	 OrderedDict([('u', 14), ('cx', 10), ('unitary', 4)])

Decomposition correct after decomposing the two qubit gates?:  True
Resulting gatecount after decomposing the two qubit gates to generic single qubit gates (u3) and CNOT (cx):
	u3: 40
	cx: 19


### Gatecounts for different gatesets:

In [67]:
# Some different (native) gatesets to play with:
gateset_heron =  ["cz", "rz", "sx", "x"]
gateset_falcon =  ["cx", "rz", "sx", "x"]
gateset_eagle =  ["ecr", "rz", "sx", "x"]
gateset_rigetti = ["rx", "rz", "cphase", "cz", "xy"]
gateset_general =  ["cz", "cx", "rx", "ry", "h", "rz", "x", "z"] 
gateset_cz =  ["cz", "rx", "ry", "h", "rz", "x", "z"]
gateset_cx =  ["cx", "rx", "ry", "h", "rz", "x", "z"]

# good for printing the circuit and to see the structure of the decomposition
gateset_structure = ["u3", "cx"]


basis_gates=gateset_cx
# Transpilation to CNOT and single qubit gates:
circ_trans = transpile(circ_decomposed, basis_gates=basis_gates, optimization_level=0)
# print(circ_trans)
print("\nNumber of gates after transpilation to gateset:", basis_gates)
pretty_print_dict(circ_trans.count_ops())

print("Decomposition correct after transpilation?: ", np.allclose(Operator(circ_trans).data, matrix_init))



Number of gates after transpilation to gateset: ['cx', 'rx', 'ry', 'h', 'rz', 'x', 'z']
	rz: 90
	rx: 28
	ry: 26
	cx: 19
Decomposition correct after transpilation?:  True


### Gatecounts after Qiskit optimization
Gatecounts after optimizing the result from the decomposition at each of the Qiskit optimization levels. Tends to reduce the number of 1-qubit gates needed (but not the number of 2-qubit gates)

For optimization levels 2 and 3 we need to manually adjust for the global phase to check the correctness of the decomposition (Qiskit does not keep track of the global phase for these)

In [70]:
# Sometimes the Qiskit optimization techniques manage to reduce the gatecount even further:
circ_qiskit_optimized_1 = transpile(circ_decomposed, basis_gates=basis_gates, optimization_level=1)
circ_qiskit_optimized_2 = transpile(circ_decomposed, basis_gates=basis_gates, optimization_level=2)
circ_qiskit_optimized_3 = transpile(circ_decomposed, basis_gates=basis_gates, optimization_level=3)

print("Gatecount after Qiskit optimization level 0 (no optimization):\t", sorted(circ_trans.count_ops().items()))
print("Gatecount after Qiskit optimization level 1:\t\t\t", sorted(circ_qiskit_optimized_1.count_ops().items()))
print("Gatecount after Qiskit optimization level 2:\t\t\t", sorted(circ_qiskit_optimized_2.count_ops().items()))
print("Gatecount after Qiskit optimization level 3:\t\t\t", sorted(circ_qiskit_optimized_3.count_ops().items()))

matrix_opt_2 = Operator(circ_qiskit_optimized_2).data 
if np.allclose(matrix_opt_2 , matrix_init):
      print("\nDecomposition correct after optimization level 2 without needing to manually account for the global phase. (which means the global phase is 0)")
else:
      print("\nNeed to manually account for phase at optimization level 2, because Qiskit does not keep track of global phase at this optimization level:")
      print("\tDecomposition correct after optimization level 2 after manually adjusting the global phase?: ", 
      np.allclose(np.exp(1j*(circ_qiskit_optimized_1.global_phase))*matrix_opt_2, matrix_init, rtol=1e-2, atol=1e-2))
      print("\tGlobal phase is: ", circ_qiskit_optimized_1.global_phase, "radians")

matrix_opt_3 = Operator(circ_qiskit_optimized_2).data 

if np.allclose(matrix_opt_3 , matrix_init):
      print("\nDecomposition correct after optimization level 3 without needing to manually account for the global phase. (which means the global phase is 0)")
else:
      print("\nNeed to manually account for phase at optimization level 3, because Qiskit does not keep track of global phase at this optimization level:")
      print("\tDecomposition correct after optimization level 3 after manually adjusting the global phase?: ",       
      np.allclose(matrix_init[0,0]/matrix_opt_3[0,0] * matrix_opt_3, matrix_init))
      print("\tGlobal phase is: ", np.arctan((1j*matrix_init[0,0]/matrix_opt_3[0,0]).real), "radians")


Gatecount after Qiskit optimization level 0 (no optimization):	 [('cx', 19), ('rx', 28), ('ry', 26), ('rz', 90)]
Gatecount after Qiskit optimization level 1:			 [('cx', 19), ('rx', 1), ('ry', 27), ('rz', 59)]
Gatecount after Qiskit optimization level 2:			 [('cx', 19), ('rx', 1), ('ry', 27), ('rz', 49)]
Gatecount after Qiskit optimization level 3:			 [('cx', 19), ('rx', 1), ('ry', 27), ('rz', 49)]

Need to manually account for phase at optimization level 2, because Qiskit does not keep track of global phase at this optimization level:
	Decomposition correct after optimization level 2 after manually adjusting the global phase?:  True
	Global phase is:  4.516659205968679 radians

Need to manually account for phase at optimization level 3, because Qiskit does not keep track of global phase at this optimization level:
	Decomposition correct after optimization level 3 after manually adjusting the global phase?:  True
	Global phase is:  0.7757594393995989 radians
